In [1]:
import torch
import os
import random
import numpy as np
import torch
from sklearn import preprocessing
import multiprocessing as mp
import torch
from torch.utils.data import DataLoader
import copy
from torch import nn
%matplotlib inline
import matplotlib.pyplot as plt
from dnaDataloader import expermentDataloader
from dnaDataloader import addData
from scipy import stats as st
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")
mp.set_start_method('spawn')
batch_size = 25
device

device(type='cuda', index=1)

## Eval with small traing sets
The goal here is to eval the model while training on small sets of data. That is how well can MLP learn from 100-200 examples of single-molecule experiments.
A verity of samples have been provided anf preprocessed (converted from excel to csv and had empty frames added) to the folder <pre>/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted</pre>

In [2]:
folders = [d[0] for d in os.walk("/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted")][1:] # remove first one is it is "/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted"
len(folders)

6

In [3]:
datasets = []
featIn = 0
for d in folders:
    data = expermentDataloader(
        f"{d}/index.csv",
        f"{d}", 
    )
    rawData = [d for d in data]
    featIn = len(rawData[0][0])
    trainValidData = []
    testData = []
    addData(testData, trainValidData, rawData, rhsSize=300)


    np.random.shuffle(trainValidData)
    trainData = []
    validData = []
    addData(trainData, validData, trainValidData, rhsSize=int(len(trainValidData)*(1/3)))

    datasets.append({"name": f"{os.path.basename(d)}", 
                     "train":DataLoader(trainData, batch_size=batch_size, shuffle=True) , 
                     "valid":DataLoader(validData, batch_size=batch_size, shuffle=True) , 
                     "test":DataLoader(testData, batch_size=len(testData), shuffle=True) ,
                     "error_margin": None
                     "model": {}}
                    )

In [4]:
print(f"datasets:")
for d in datasets:
    print(d)
print(f"{len(datasets)}")
print(f"featIn: {featIn}")

datasets:
{'name': '1800_nM_AR_out', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffd53490>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffc4b850>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffadde70>, 'model': {}}
{'name': '800_nM_AR_out', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffd53be0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffa8ff70>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffaf6560>, 'model': {}}
{'name': '1200_nM_AR_out', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f1e0590bca0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffaf4040>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffb2feb0>, 'model': {}}
{'name': '400_nM_AR_out', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffaf7f70>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f1dffb2c430>, 'test': <tor

In [5]:
for d in datasets:
    print(f"-- {d['name']} --")
    print(f"train: {len(d['train'])}")
    print(f"valid: {len(d['valid'])}")
    print(f"test : {len(d['test'])}")

-- 1800_nM_AR_out --
train: 8
valid: 4
test : 1
-- 800_nM_AR_out --
train: 8
valid: 4
test : 1
-- 1200_nM_AR_out --
train: 8
valid: 4
test : 1
-- 400_nM_AR_out --
train: 8
valid: 4
test : 1
-- 50_nM_AR_out --
train: 8
valid: 4
test : 1
-- 100_nM_AR_out --
train: 8
valid: 4
test : 1


In [6]:
manager = mp.Manager()
return_dict = manager.dict()

In [7]:
from train_def import train

In [8]:
processes = []
devices = [torch.device("cuda:0"),torch.device("cuda:1"),torch.device("cuda:2"),torch.device("cuda:3")]
for d in datasets:
    processes.append(mp.Process(target=train, args=(d["train"], d["test"], d["name"], featIn, return_dict, 60, devices[0])))
    devices.append(devices.pop(0))
    
processes

[<Process name='Process-2' parent=1041680 initial>,
 <Process name='Process-3' parent=1041680 initial>,
 <Process name='Process-4' parent=1041680 initial>,
 <Process name='Process-5' parent=1041680 initial>,
 <Process name='Process-6' parent=1041680 initial>,
 <Process name='Process-7' parent=1041680 initial>]

In [9]:
print(return_dict)

{}


In [10]:
processesList = list(range(len(processes)))

while processesList:
    run = processesList[:4]
    processesList = processesList[4:]
    for i in run:
        processes[i].start()
    for i in run:
        processes[i].join()
        processes[i].terminate()
print(return_dict)

training 1800_nM_AR_out...
training 800_nM_AR_out...
training 1200_nM_AR_out...
training 400_nM_AR_out...
training 50_nM_AR_out...
training 100_nM_AR_out...
{'1800_nM_AR_out': {'path': './Models/smallTrain/1800_nM_AR_out_18.05.2023_16-51-02-722778.pt', 'acc': 0.37244897959183676}, '800_nM_AR_out': {'path': './Models/smallTrain/800_nM_AR_out_18.05.2023_16-51-03-577436.pt', 'acc': 0.10606060606060606}, '1200_nM_AR_out': {'path': './Models/smallTrain/1200_nM_AR_out_18.05.2023_16-51-04-609774.pt', 'acc': 0.3765182186234818}, '400_nM_AR_out': {'path': './Models/smallTrain/400_nM_AR_out_18.05.2023_16-51-05-649048.pt', 'acc': 0.2655367231638418}, '50_nM_AR_out': {'path': './Models/smallTrain/50_nM_AR_out_18.05.2023_16-51-10-168149.pt', 'acc': 0.75}, '100_nM_AR_out': {'path': './Models/smallTrain/100_nM_AR_out_18.05.2023_16-51-11-036066.pt', 'acc': 0.3417085427135678}}
